In [4]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.appName('some-spark').config('master','local').getOrCreate()
spark.sql('select 1').show()

+---+
|  1|
+---+
|  1|
+---+



In [3]:
df_logs = spark.read.format("csv").option("header", "true").option("inferSchema","true").load("web_server_logs.csv")
df_logs.show()

+---------------+--------------------+------+--------------------+-------------+-------------+
|             ip|           timestamp|method|                 url|response_code|response_size|
+---------------+--------------------+------+--------------------+-------------+-------------+
|  33.165.201.68|2025-07-22 02:07:...|  POST|            main/app|          301|         4896|
| 149.11.111.150|2025-06-14 19:50:...|   PUT|           list/list|          200|         4642|
| 98.126.255.199|2025-06-02 01:36:...|   PUT|                 app|          404|         2159|
|  102.14.20.117|2025-10-13 12:15:...|   GET|    wp-content/posts|          500|          267|
| 145.90.182.161|2025-03-02 11:57:...|   GET| list/wp-content/tag|          500|         4999|
| 82.236.204.213|2025-06-15 16:49:...|  POST|                 app|          301|          588|
| 13.101.193.248|2025-07-13 02:28:...|   PUT| tags/explore/search|          200|         2850|
| 220.193.237.85|2025-03-22 16:03:...|  POST|     

In [8]:
print("Самые активные")
df_logs.groupby(F.col("ip")).agg(F.count(F.col("url")).alias("request_count")).orderBy(F.col("request_count").desc()).show(10)

Самые активные
+---------------+-------------+
|             ip|request_count|
+---------------+-------------+
|118.199.168.128|            2|
|  222.196.13.62|            1|
|  139.24.12.236|            1|
|   23.38.109.57|            1|
|  69.141.52.204|            1|
|  154.142.7.206|            1|
| 10.217.174.170|            1|
|181.163.247.197|            1|
|  204.97.151.16|            1|
| 96.111.160.213|            1|
+---------------+-------------+
only showing top 10 rows



In [9]:
print("Кол-во запросов по каждому методу")
df_logs.groupby(F.col("method")).agg(F.count(F.col("url"))).show()

+------+----------+
|method|count(url)|
+------+----------+
|  POST|     24972|
|DELETE|     25062|
|   PUT|     24879|
|   GET|     25087|
+------+----------+



In [13]:
print("Кол-во запросов с кодом ответа 404: ")
df_logs.filter(F.col("response_code") == '404').count()

Кол-во запросов с кодом ответа 404: 


24872

In [16]:
print("Total response size by day")
df_logs.groupby(F.to_date(F.col("timestamp")).alias("date")).agg(F.sum(F.col("response_size")).alias("total_response_size")).orderBy(F.col("date")).show()

Кол-во запросов по каждому методу
+----------+-------------------+
|      date|total_response_size|
+----------+-------------------+
|2025-01-01|            1808230|
|2025-01-02|            1791636|
|2025-01-03|            1596052|
|2025-01-04|            1562972|
|2025-01-05|            1693638|
|2025-01-06|            1809918|
|2025-01-07|            1853817|
|2025-01-08|            1670941|
|2025-01-09|            1774837|
|2025-01-10|            1853071|
|2025-01-11|            1688328|
|2025-01-12|            1750350|
|2025-01-13|            1657092|
|2025-01-14|            1789343|
|2025-01-15|            1642485|
|2025-01-16|            1622700|
|2025-01-17|            1570490|
|2025-01-18|            1777843|
|2025-01-19|            1688751|
|2025-01-20|            1582007|
+----------+-------------------+
only showing top 20 rows

